In [64]:
import os
import moviepy.editor as mpy
from moviepy.video.tools.segmenting import findObjects

IMAGEMAGICK_BINARY = os.getenv ('IMAGEMAGICK_BINARY', 'C:\\Program Files\\ImageMagick\\magick.exe')
FFMPEG_BINARY = os.getenv('FFMPEG_BINARY', 'ffmpeg-imageio')

# print(mpy.TextClip.list("font"))


In [65]:
SCREEN_WIDTH = int(720)
SCREEN_HEIGHT = int(480)
WHITE = (255, 255, 255)
BACKGROUND_HEX = "#007EB6"
BACKGROUND_RGB = (0, 126, 182)
SCREEN_SIZE = (SCREEN_WIDTH, SCREEN_HEIGHT)
VERTICAL_SPACE=30
HORIZONTAL_SPACE=100
DISPLAY_HEIGHT = int(SCREEN_HEIGHT*0.6)
DISPLAY_WIDTH = int(SCREEN_WIDTH*0.8)
DISPLAY_SIZE = (DISPLAY_WIDTH, DISPLAY_HEIGHT)
TEXT_HEIGHT = int(DISPLAY_HEIGHT / 4)
TEXT_WIDTH = int(DISPLAY_WIDTH)
TEXT_SIZE = (TEXT_WIDTH, TEXT_HEIGHT)
MARGIN_LEFT = int(SCREEN_WIDTH*0.1)
MARGIN_TOP = int(SCREEN_WIDTH*0.2)

COLOR_PREV = "rgba(255, 255, 255, 0.8)"
COLOR_CUR = "rgba(255, 255, 255, 1)"
COLOR_NEXT = "rgba(0, 0, 0, 1)"

# final_clip.write_videofile("video_with_python.mp4", fps=10)

'And spring became the summer'

In [69]:
def create_mp4(lrcfilepath: str, audiofilepath: str, outputfilepath: str):
    def timestamp_converter(timestamp_str: str):
        # example timestamp_str: 01:14.79 
        mins, secs = [float(x) for x in timestamp_str.split(":")]
        timestamp_ms = int(secs * 1000 + mins * 60000)

        return timestamp_ms

    def read_lrc(filepath: str):
        lyrics = []
        with open(filepath, "r", encoding="utf8") as file:
            for line in file.read().splitlines():
                timestamp_str = line[1:9]
                timestamp_ms = timestamp_converter(timestamp_str)
                lyric = line[11:]
                
                if lyric == "": lyric = " "

                lyrics.append((timestamp_ms, lyric))
        return lyrics

    lyrics = read_lrc(lrcfilepath)    
    
    clips = []

    audio = mpy.AudioFileClip(audiofilepath)
    audioClip = mpy.CompositeAudioClip([audio])

    totalDuration = audioClip.duration

    einsatz_cur = 0

    BACKGROUND_CLIP = mpy.ColorClip(size=SCREEN_SIZE, color=BACKGROUND_RGB)

    for i in range(-1, len(lyrics)):

        z = min(max(1, i), len(lyrics)-3)

        previous_text = lyrics[z-1][1]
        current_text = lyrics[z][1]
        next_text = lyrics[z+1][1]
        nextt_text = lyrics[z+2][1]

        einsatz_next = lyrics[min(i+1, len(lyrics)-1)][0]

        screentime = einsatz_next - einsatz_cur  
        einsatz_cur = einsatz_next

        if i == len(lyrics):
            screentime = einsatz_cur - totalDuration


        colors = []


        if i == -1:
            colors = [COLOR_NEXT for _ in range(4)]
        elif i == 0:
            colors = [COLOR_CUR, COLOR_NEXT, COLOR_NEXT, COLOR_NEXT]
        elif i == len(lyrics) - 2:
            colors = [COLOR_PREV, COLOR_PREV, COLOR_CUR, COLOR_NEXT]
        elif i == len(lyrics) - 1:
            colors = [COLOR_PREV, COLOR_PREV, COLOR_PREV, COLOR_CUR]
        elif i >= len(lyrics):
            colors = [COLOR_PREV for _ in range(4)]
        else:
            colors = [COLOR_PREV, COLOR_CUR, COLOR_NEXT, COLOR_NEXT]


        texts = [previous_text, current_text, next_text, nextt_text]

        lineclips = [BACKGROUND_CLIP]

        for i, line in enumerate(texts):
            lineclip = mpy.TextClip(
                    line,
                    color=colors[i],
                    size=TEXT_SIZE,          
                    align="center",
                    font="Radio-Medium",
                    fontsize=32
                ).set_position((MARGIN_LEFT, i*TEXT_HEIGHT+MARGIN_TOP))
            lineclips.append(lineclip)

        clips.append(
            mpy.CompositeVideoClip(lineclips, size=SCREEN_SIZE).set_duration(screentime/1000)
        )

    mpy.concatenate_videoclips(clips, method="chain").on_color(color=BACKGROUND_RGB).write_videofile(outputfilepath, fps=5, audio=audiofilepath)

In [68]:
create_mp4("./test_batch/01 Sweet Caroline.lrc", "./cutted/01 Sweet Caroline2.mp3", "final.mp4")

                                                                
                                                                
t:   0%|          | 2/972 [10:04<81:30:21, 302.50s/it, now=None]

Moviepy - Building video video.mp4.
Moviepy - Writing video video.mp4








































































































































































































































































































































































































































































                                                                
                                                                
t:   0%|          | 2/972 [10:58<88:44:18, 329.34s/it, now=None]

Moviepy - Done !
Moviepy - video ready video.mp4
